## Next Steps

1. Plot Output Video
2. Create Ongoing Calculations
3. Record Calculations to Output Video
4. Machine Learning for how to win

### Import and Roughly Clean Data

In [1]:
pip list

Package                       Version
----------------------------- ------------------
absl-py                       1.3.0
antlr4-python3-runtime        4.9.3
appnope                       0.1.3
astor                         0.8.1
attrs                         22.2.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
black                         22.12.0
Bottleneck                    1.3.5
brotlipy                      0.7.0
certifi                       2022.12.7
cffi                          1.15.1
chardet                       5.1.0
charset-normalizer            3.0.1
click                         8.0.4
cloudpickle                   2.2.1
cryptography                  38.0.4
cycler                        0.11.0
Cython                        0.29.33
dataclasses                   0.6
debugpy                       1.5.1
decorator                     4.4.2
Deprecated                    1.2.13
emoji                         2.2.0
entrypoints                   0.4
ffmp

In [40]:
import pandas as pd
import cv2
import numpy as np

out_fps = 29.3

run = 15
path = f"exp{run}/labels/Football_intersquad.txt"
df = pd.read_csv(path, header=None, sep=" ", names=["frame", "class", "ID", "bb_left", "bb_top", "bb_w", "bb_h", "None1", "None2", "None3", "None4", "NaN"])
df['x_mid'] = df['bb_left'] + (df['bb_w'] / 2)
df['y_mid'] = df['bb_top'] + (df['bb_h'] / 2)
df = df[['frame', 'class', 'ID', 'x_mid', 'y_mid']].copy()
df = df.loc[(df['class']==0) | (df['class']==32)] #Person & Sports Ball

In [41]:
df.shape

(10842, 5)

In [42]:
df['frame'].nunique()

548

### Identify Players

In [43]:
for frame in df['frame'].unique():
    print(frame, df[(df['frame'] == frame) & (df['class'] == 0)]['ID'].unique())

3 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
4 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
5 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
6 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
7 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 22]
8 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 22]
9 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 22]
10 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 22]
11 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 22]
12 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 22]
13 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 22]
14 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 22]
15 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
16 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 22]
17 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 22]
18 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 22]
19 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 22]
20 [ 1  2  3  4  5 

In [44]:
players = {1:"player1",
           2:"player2",
           3:"player3",
           4:"player4",
           5:"Referee",
            6:"player5",
            7:"player6",
            8:"player7",
            9:"Eric",
            10:"player8",
            11:"player9",
            12:"player10",
            13:"player11",
            14:"player12",
            15:"player18",
            16:"player14",
            17:"player13",
            22:"player19",
            33:"player15",
            37:"player16",
            43:"player1",
            54:"player17",
            78:"player1", # & when player1 change to 78, player14 changes to 1
            162:"player14",
            172:"player3",
           }
team = {1:"F",
        2:"F",
        3:"H",
        4:"H",
        5:"R",
        6:"F",
        7:"H",
        8:"F",
        9:"F",
        10:"F",
        11:"H",
        12:"H",
        13:"H",
        14:"H",
        15:"F",
        16:"F",
        17:"F",
        22:"F",
        33:"H",
        37:"F",
        43:"F",
        54:"H",
        78:"F",
        162:"F",
        172:"H",
        }
players = pd.DataFrame.from_dict(players, orient='index', columns=['Name']).reset_index().rename(columns={"index":"ID"})
team = pd.DataFrame.from_dict(team, orient='index', columns=['Team']).reset_index().rename(columns={"index":"ID"})

#### Clean Players

In [45]:
df = pd.merge(df, players, how='left', on='ID')

In [46]:
df = pd.merge(df, team, how='left', on='ID')

In [47]:
df.loc[(df['frame'] >= df[df['ID']==78]['frame'].min()) & (df['ID'] == 1), "Name"] = "player14"

In [48]:
df = df.drop(df[(df['frame'].isin([45,46,47,48,49,50,57])) & (df['ID'] == 1)].index)

In [49]:
df = df.drop(df[(df['frame'].isin([117, 120, 124, 128, 132, 136, 163, 166, 171, 176, 178])) & (df['ID'] == 1)].index)

In [50]:
df

,frame,class,ID,x_mid,y_mid,Name,Team
0,3,0,1,3622.0,1945.0,player1,F
1,3,0,2,3629.5,1262.5,player2,F
2,3,0,3,3093.5,1775.5,player3,H
3,3,0,4,1163.0,468.5,player4,H
4,3,0,5,1354.5,608.5,Referee,R
...,...,...,...,...,...,...,...
10837,550,0,33,1091.0,476.5,player15,H
10838,550,0,37,1418.0,116.0,player16,F
10839,550,0,54,3095.5,274.0,player17,H
10840,550,0,78,1464.5,526.5,player1,F


In [51]:
clean = df[['frame', 'Name']].value_counts()

In [52]:
clean.reset_index()[clean.reset_index()[0]>1].sort_values(by='frame')

,frame,Name,0


### Identify Ball

In [53]:
df.loc[df['class'] == 32, "Name"] = "ball"
df.loc[df['class'] == 32, "Team"] = "N"

### Check any remaining NANs

In [54]:
df[df.isna().any(axis=1)]

,frame,class,ID,x_mid,y_mid,Name,Team
3750,182,0,111,18.5,1483.0,NaN,NaN
3772,183,0,111,16.5,1485.0,NaN,NaN


In [55]:
df = df.drop(df[(df['ID'] == 111)].index)

In [56]:
# Extraneous Sports Ball
df = df.drop(df[(df['ID'] == 38)].index)

In [57]:
df[df.isna().any(axis=1)]

,frame,class,ID,x_mid,y_mid,Name,Team


### Interpolate Locations

In [58]:
df

,frame,class,ID,x_mid,y_mid,Name,Team
0,3,0,1,3622.0,1945.0,player1,F
1,3,0,2,3629.5,1262.5,player2,F
2,3,0,3,3093.5,1775.5,player3,H
3,3,0,4,1163.0,468.5,player4,H
4,3,0,5,1354.5,608.5,Referee,R
...,...,...,...,...,...,...,...
10837,550,0,33,1091.0,476.5,player15,H
10838,550,0,37,1418.0,116.0,player16,F
10839,550,0,54,3095.5,274.0,player17,H
10840,550,0,78,1464.5,526.5,player1,F


In [59]:
list(players.Name.unique()) + ['ball']

['player1',
 'player2',
 'player3',
 'player4',
 'Referee',
 'player5',
 'player6',
 'player7',
 'Eric',
 'player8',
 'player9',
 'player10',
 'player11',
 'player12',
 'player18',
 'player14',
 'player13',
 'player19',
 'player15',
 'player16',
 'player17',
 'ball']

In [60]:
outer_df = []
for frame in df['frame'].unique():
    for name in list(players.Name.unique()) + ['ball']:
        outer_df.append({"frame":frame, "Name":name})
outer_df = pd.DataFrame(outer_df)

In [61]:
outer_df

,frame,Name
0,3,player1
1,3,player2
2,3,player3
3,3,player4
4,3,Referee
...,...,...
12051,550,player19
12052,550,player15
12053,550,player16
12054,550,player17


In [62]:
df = pd.merge(df,outer_df, on=['frame', 'Name'], how='outer')

In [63]:
df = df.sort_values(['frame', 'Name']).reset_index(drop=True)

In [64]:
df[df['Name']=="player3"].head(50)

,frame,class,ID,x_mid,y_mid,Name,Team
15,3,0.0,3.0,3093.5,1775.5,player3,H
37,4,0.0,3.0,3090.5,1786.0,player3,H
59,5,0.0,3.0,3086.5,1796.0,player3,H
81,6,0.0,3.0,3086.5,1806.0,player3,H
103,7,0.0,3.0,3085.5,1815.0,player3,H
...,...,...,...,...,...,...,...
1005,48,NaN,NaN,NaN,NaN,player3,NaN
1027,49,NaN,NaN,NaN,NaN,player3,NaN
1049,50,0.0,3.0,2951.0,1698.5,player3,H
1071,51,0.0,3.0,2939.0,1692.0,player3,H


In [27]:
# Ball Missing at end of the video, so let's add in a couple manual locations to help interpolate
# The ball is no longer tracked once player13 catches it, so let's set all of those values equal to player13's -10

pd.set_option('display.max_rows', 500)

df[df['Name'] == "ball"].tail(200)


,frame,class,ID,x_mid,y_mid,Name,Team
7706,355,NaN,NaN,NaN,NaN,ball,NaN
7728,356,32.0,137.0,921.5,467.5,ball,N
7750,357,32.0,137.0,924.0,455.0,ball,N
7772,358,32.0,137.0,928.5,442.0,ball,N
7794,359,32.0,137.0,933.0,429.5,ball,N
7816,360,32.0,137.0,937.5,418.5,ball,N
7838,361,32.0,137.0,941.5,407.5,ball,N
7860,362,32.0,137.0,946.0,397.0,ball,N
7882,363,32.0,137.0,951.5,386.5,ball,N
7904,364,32.0,137.0,956.5,378.5,ball,N


In [65]:
pd.set_option('display.max_rows', 10)
for frame in range(401, df['frame'].max()):
    # print(frame)
    df.loc[(df['Name'] == "ball") & (df['frame'] == frame), 'x_mid'] = (df.loc[(df['Name'] == "player13") & (df['frame'] == frame), 'x_mid'] - 10).values
    df.loc[(df['Name'] == "ball") & (df['frame'] == frame), 'y_mid'] = (df.loc[(df['Name'] == "player13") & (df['frame'] == frame), 'y_mid'] + 20).values

In [66]:
# player7 and player9 move and stay out of the frame about half way through, so let's remove their records after their last appearance
print("player7", df[(df['Name'] == 'player7') & (df['x_mid'] == df[df['Name'] == 'player7']['x_mid'].min())]['frame'].values[0])
print("player9", df[(df['Name'] == 'player9') & (df['x_mid'] == df[df['Name'] == 'player9']['x_mid'].min())]['frame'].values[0])

M_frame = df[(df['Name'] == 'player7') & (df['x_mid'] == df[df['Name'] == 'player7']['x_mid'].min())]['frame'].values[0]
S_frame = df[(df['Name'] == 'player9') & (df['x_mid'] == df[df['Name'] == 'player9']['x_mid'].min())]['frame'].values[0]
df = df.drop(df[(df['frame'] > M_frame) & (df['Name'] == "player7")].index)
df = df.drop(df[(df['frame'] > S_frame) & (df['Name'] == "player9")].index)

player7 181
player9 214


In [67]:
outdf = []

for Name, group in df.groupby("Name"):
    group['x_mid'] = group['x_mid'].interpolate()
    group['y_mid'] = group['y_mid'].interpolate()
    outdf.append(group)
df = pd.concat(outdf)

In [68]:
df[df['Name']=="player3"].head(50)

,frame,class,ID,x_mid,y_mid,Name,Team
15,3,0.0,3.0,3093.500000,1775.500000,player3,H
37,4,0.0,3.0,3090.500000,1786.000000,player3,H
59,5,0.0,3.0,3086.500000,1796.000000,player3,H
81,6,0.0,3.0,3086.500000,1806.000000,player3,H
103,7,0.0,3.0,3085.500000,1815.000000,player3,H
...,...,...,...,...,...,...,...
1005,48,NaN,NaN,2970.428571,1706.928571,player3,NaN
1027,49,NaN,NaN,2960.714286,1702.714286,player3,NaN
1049,50,0.0,3.0,2951.000000,1698.500000,player3,H
1071,51,0.0,3.0,2939.000000,1692.000000,player3,H


In [69]:
df.columns

Index(['frame', 'class', 'ID', 'x_mid', 'y_mid', 'Name', 'Team'], dtype='object')

In [70]:
name_team = df[['Name', "Team"]].drop_duplicates().dropna()

In [71]:
df.drop(columns=['Team'], inplace=True)
df = pd.merge(df, name_team, on=['Name'], how='left')

In [72]:
df[df['Name']=="player3"].head(50)

,frame,class,ID,x_mid,y_mid,Name,Team
8226,3,0.0,3.0,3093.500000,1775.500000,player3,H
8227,4,0.0,3.0,3090.500000,1786.000000,player3,H
8228,5,0.0,3.0,3086.500000,1796.000000,player3,H
8229,6,0.0,3.0,3086.500000,1806.000000,player3,H
8230,7,0.0,3.0,3085.500000,1815.000000,player3,H
...,...,...,...,...,...,...,...
8271,48,NaN,NaN,2970.428571,1706.928571,player3,H
8272,49,NaN,NaN,2960.714286,1702.714286,player3,H
8273,50,0.0,3.0,2951.000000,1698.500000,player3,H
8274,51,0.0,3.0,2939.000000,1692.000000,player3,H


In [73]:
df = df.drop(columns=['class', 'ID'])

### Begin Calculations

In [74]:
quad_coords = {
    "lonlat": np.array([
        [41.709011, -72.585549], # Top Right
        [41.709691, -72.585688], # Top left
        [41.709500, -72.586105], # Bottom Left
        [41.709091, -72.586046] # Bottom Right
    ]),
    "pixel": np.array([
        [3697, 264], # top right
        [95, 273], # top left
        [19, 2137], # bottom left
        [3775, 2116] # bottom right
    ])
}

In [75]:
import numpy as np
import cv2


class PixelMapper(object):
    """
    Create an object for converting pixels to geographic coordinates,
    using four points with known locations which form a quadrilteral in both planes
    Parameters
    ----------
    pixel_array : (4,2) shape numpy array
        The (x,y) pixel coordinates corresponding to the top left, top right, bottom right, bottom left
        pixels of the known region
    lonlat_array : (4,2) shape numpy array
        The (lon, lat) coordinates corresponding to the top left, top right, bottom right, bottom left
        pixels of the known region
    """
    def __init__(self, pixel_array, lonlat_array):
        assert pixel_array.shape==(4,2), "Need (4,2) input array"
        assert lonlat_array.shape==(4,2), "Need (4,2) input array"
        self.M = cv2.getPerspectiveTransform(np.float32(pixel_array),np.float32(lonlat_array))
        self.invM = cv2.getPerspectiveTransform(np.float32(lonlat_array),np.float32(pixel_array))
        
    def pixel_to_lonlat(self, pixel):
        """
        Convert a set of pixel coordinates to lon-lat coordinates
        Parameters
        ----------
        pixel : (N,2) numpy array or (x,y) tuple
            The (x,y) pixel coordinates to be converted
        Returns
        -------
        (N,2) numpy array
            The corresponding (lon, lat) coordinates
        """
        if type(pixel) != np.ndarray:
            pixel = np.array(pixel).reshape(1,2)
        assert pixel.shape[1]==2, "Need (N,2) input array" 
        pixel = np.concatenate([pixel, np.ones((pixel.shape[0],1))], axis=1)
        lonlat = np.dot(self.M,pixel.T)
        
        return (lonlat[:2,:]/lonlat[2,:]).T
    
    def lonlat_to_pixel(self, lonlat):
        """
        Convert a set of lon-lat coordinates to pixel coordinates
        Parameters
        ----------
        lonlat : (N,2) numpy array or (x,y) tuple
            The (lon,lat) coordinates to be converted
        Returns
        -------
        (N,2) numpy array
            The corresponding (x, y) pixel coordinates
        """
        if type(lonlat) != np.ndarray:
            lonlat = np.array(lonlat).reshape(1,2)
        assert lonlat.shape[1]==2, "Need (N,2) input array" 
        lonlat = np.concatenate([lonlat, np.ones((lonlat.shape[0],1))], axis=1)
        pixel = np.dot(self.invM,lonlat.T)
        
        return (pixel[:2,:]/pixel[2,:]).T

In [76]:
pm = PixelMapper(quad_coords["pixel"], quad_coords["lonlat"])

In [77]:
pm.lonlat_to_pixel((41.709011, -72.585549))

array([[3690.368797  ,  266.03596056]])

In [78]:
pm.pixel_to_lonlat((3690.368797  ,  266.03596056))

array([[ 41.70901241, -72.5855491 ]])

In [79]:
df['lon_lat'] = df.apply(lambda x: pm.pixel_to_lonlat((x['x_mid'], x['y_mid']))[0], axis=1)

In [80]:
df['lon'] = df['lon_lat'].str[0]
df['lat'] = df['lon_lat'].str[1]

In [81]:
import haversine as hs
from haversine import Unit

outdf = []
for Name, group in df.groupby("Name"):
    print(Name)
    group['lon_prev'] = group['lon'].shift(1)
    group['lat_prev'] = group['lat'].shift(1)
    group['haversine'] = group.apply(lambda x: hs.haversine((x['lat'], x['lon']), (x['lat_prev'], x['lon_prev']), unit=Unit.MILES), axis=1)
    group['haversine_smooth'] = group['haversine'].rolling(window=5).mean()
    group['haversine_smooth'] = group['haversine'].rolling(window=29).mean()
    outdf.append(group)
df = pd.concat(outdf).sort_index()

Eric
Referee
ball
player1
player10
player11
player12
player13
player14
player15
player16
player17
player18
player19
player2
player3
player4
player5
player6
player7
player8
player9


#### Derive Insights

In [82]:
df['mph'] = df['haversine'] * 29 * 3600
df['mph_smoothed'] = df['haversine_smooth'] * 29 * 3600

In [83]:
df.dropna()[~df['Name'].isin(["ball"])].sort_values('mph_smoothed').tail(50) #[~df['Name'].isin(["player1", "player14", "player3"])] .dropna()[~df['Name'].isin(["ball"])]

/Users/ericstassen/opt/anaconda3/envs/vision37/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,haversine,haversine_smooth,mph,mph_smoothed
4522,135,1499.00,1067.50,player14,F,"[41.70938520354147, -72.5858872547701]",41.709385,-72.585887,41.709385,-72.585885,0.000171,0.000146,17.830438,15.191707
1764,117,2328.00,1023.00,player1,F,"[41.70926701577826, -72.58585626568639]",41.709267,-72.585856,41.709267,-72.585858,0.000109,0.000146,11.342549,15.195154
1771,124,2319.50,965.00,player1,F,"[41.709269227306386, -72.58584132400034]",41.709269,-72.585841,41.709269,-72.585844,0.000219,0.000146,22.855664,15.219847
4524,137,1496.00,1079.50,player14,F,"[41.7093850161835, -72.58589019751832]",41.709385,-72.585890,41.709385,-72.585889,0.000074,0.000146,7.767997,15.246432
1775,128,2310.75,936.75,player1,F,"[41.70927103163864, -72.58583396686325]",41.709271,-72.585834,41.709271,-72.585836,0.000115,0.000146,11.965899,15.252367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4532,145,1459.00,1167.50,player14,F,"[41.709385740883754, -72.58591147337117]",41.709386,-72.585911,41.709386,-72.585909,0.000164,0.000165,17.165388,17.175529
4538,151,1431.00,1219.50,player14,F,"[41.70938698010694, -72.58592366082419]",41.709387,-72.585924,41.709387,-72.585922,0.000106,0.000165,11.015962,17.210588
4537,150,1435.50,1213.00,player14,F,"[41.70938669200585, -72.5859221360977]",41.709387,-72.585922,41.709386,-72.585920,0.000153,0.000165,15.992233,17.249903
4540,153,1419.50,1237.00,player14,F,"[41.70938766757313, -72.58592772323817]",41.709388,-72.585928,41.709387,-72.585925,0.000198,0.000166,20.638395,17.365880


In [84]:
df.dropna()[df['Name'].isin(["ball"])].sort_values('mph_smoothed').tail(50)

/Users/ericstassen/opt/anaconda3/envs/vision37/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,haversine,haversine_smooth,mph,mph_smoothed
1481,384,1083.5,328.0,ball,N,"[41.709508230467264, -72.58567311847416]",41.709508,-72.585673,41.709510,-72.585672,0.000060,0.001749,6.214173,182.559853
1490,392,1131.0,320.5,ball,N,"[41.70950060468302, -72.58566854552993]",41.709501,-72.585669,41.709243,-72.585830,0.012351,0.001752,1289.397541,182.878649
1480,383,1076.0,325.5,ball,N,"[41.709509844963264, -72.58567240525413]",41.709510,-72.585672,41.709511,-72.585671,0.000072,0.001757,7.536113,183.451788
1479,382,1069.5,322.5,ball,N,"[41.70951134102616, -72.58567146131563]",41.709511,-72.585671,41.709513,-72.585671,0.000038,0.001764,4.009133,184.120366
1478,381,1062.0,321.0,ball,N,"[41.70951284654615, -72.58567113590539]",41.709513,-72.585671,41.709514,-72.585671,0.000048,0.001771,5.022231,184.903474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,366,966.5,362.0,ball,N,"[41.709524936219125, -72.58569010790286]",41.709525,-72.585690,41.709239,-72.585840,0.011948,0.003073,1247.376150,320.803095
1464,367,972.5,354.5,ball,N,"[41.7095247652148, -72.58568707512396]",41.709525,-72.585687,41.709525,-72.585690,0.000210,0.003076,21.879619,321.120225
1465,368,977.5,348.0,ball,N,"[41.70952465129598, -72.58568443887808]",41.709525,-72.585684,41.709525,-72.585687,0.000182,0.003078,19.017762,321.385910
1466,369,984.0,343.0,ball,N,"[41.709524096431295, -72.58568231133096]",41.709524,-72.585682,41.709525,-72.585684,0.000147,0.003080,15.393425,321.583751


### Ball Data was messier so let's clean up obvious errors

In [85]:
df.loc[(df['Name'] == 'ball') & (df['mph'] > 50), 'x_mid'] = np.nan
df.loc[(df['Name'] == 'ball') & (df['mph'] > 50), 'y_mid'] = np.nan

In [86]:
outdf = []

for Name, group in df.groupby("Name"):
    group['x_mid'] = group['x_mid'].interpolate()
    group['y_mid'] = group['y_mid'].interpolate()
    outdf.append(group)
df = pd.concat(outdf)

In [87]:
df['lon_lat'] = df.apply(lambda x: pm.pixel_to_lonlat((x['x_mid'], x['y_mid']))[0], axis=1)
df['lon'] = df['lon_lat'].str[0]
df['lat'] = df['lon_lat'].str[1]
outdf = []
for Name, group in df.groupby("Name"):
    print(Name)
    group['lon_prev'] = group['lon'].shift(1)
    group['lat_prev'] = group['lat'].shift(1)
    group['haversine'] = group.apply(lambda x: hs.haversine((x['lat'], x['lon']), (x['lat_prev'], x['lon_prev']), unit=Unit.MILES), axis=1)
    group['haversine_smooth'] = group['haversine'].rolling(window=5).mean()
    group['haversine_smooth'] = group['haversine'].rolling(window=29).mean()
    outdf.append(group)
df = pd.concat(outdf).sort_index()
df['mph'] = df['haversine'] * 29 * 3600
df['mph_smoothed'] = df['haversine_smooth'] * 29 * 3600

Eric
Referee
ball
player1
player10
player11
player12
player13
player14
player15
player16
player17
player18
player19
player2
player3
player4
player5
player6
player7
player8
player9


In [88]:
df.dropna()[df['Name'].isin(["ball"])].sort_values('mph_smoothed').tail(50)

/Users/ericstassen/opt/anaconda3/envs/vision37/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,haversine,haversine_smooth,mph,mph_smoothed
1476,379,1048.5,319.0,ball,N,"[41.70951547881897, -72.58567082297127]",41.709515,-72.585671,41.709517,-72.585671,0.000031,0.000178,3.270071,18.558809
1470,373,1008.0,325.5,ball,N,"[41.70952191486213, -72.58567475280624]",41.709522,-72.585675,41.709522,-72.585676,0.000106,0.000179,11.064575,18.652530
1475,378,1041.0,318.0,ball,N,"[41.70951692939156, -72.58567069240543]",41.709517,-72.585671,41.709518,-72.585671,0.000048,0.000180,4.977807,18.767819
1471,374,1014.5,323.5,ball,N,"[41.70952099699203, -72.58567375229828]",41.709521,-72.585674,41.709522,-72.585675,0.000072,0.000180,7.484096,18.773557
1474,377,1034.5,319.0,ball,N,"[41.70951796973769, -72.58567130825621]",41.709518,-72.585671,41.709519,-72.585672,0.000050,0.000181,5.171855,18.900138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1427,334,630.0,557.0,ball,N,"[41.70955818856458, -72.58576845863223]",41.709558,-72.585768,41.709559,-72.585772,0.000265,0.000433,27.613927,45.251904
1423,330,565.5,599.0,ball,N,"[41.709563447419946, -72.58578359327494]",41.709563,-72.585784,41.709564,-72.585788,0.000310,0.000434,32.349172,45.285443
1424,331,581.5,587.0,ball,N,"[41.709562347866814, -72.5857793883705]",41.709562,-72.585779,41.709563,-72.585784,0.000291,0.000439,30.424200,45.839446
1425,332,598.5,577.0,ball,N,"[41.70956083895504, -72.58577574838218]",41.709561,-72.585776,41.709562,-72.585779,0.000253,0.000440,26.457811,45.892804


Following code for distance to each other:

distance_df = []
for Frame, group in df.groupby("frame"):
    print(Frame)
    group['distances_mile'] = group.apply(lambda x: {"person_" + group.loc[index]['Name']: hs.haversine((x['lat'], x['lon']), (group.loc[index]['lat'], group.loc[index]['lon']), unit=Unit.MILES) for index in group.index}, axis=1)
    distance_df.append(group)
distance_df = pd.concat(distance_df)

In [89]:
distance_df = []
for Frame, group in df.groupby("frame"):
    print(Frame)
    group['ft_to_ball'] = group.apply(lambda x: hs.haversine((group.loc[group['Name'] == 'ball'].values[0][7], group.loc[group['Name'] == 'ball'].values[0][6]), (x['lat'], x['lon']), unit=Unit.MILES), axis=1) * 5280
    distance_df.append(group)
df = pd.concat(distance_df)

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278


In [90]:
df

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,haversine,haversine_smooth,mph,mph_smoothed,ft_to_ball
0,3,3381.5,941.0,Eric,F,"[41.70910594903356, -72.5858070933292]",41.709106,-72.585807,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,3,1354.5,608.5,Referee,R,"[41.70943594693989, -72.58576355882184]",41.709436,-72.585764,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1096,3,NaN,NaN,ball,N,"[nan, nan]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1650,3,3622.0,1945.0,player1,F,"[41.709104413070285, -72.58601833065894]",41.709104,-72.586018,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2198,3,2855.0,703.0,player10,H,"[41.70918556556278, -72.58574959387205]",41.709186,-72.585750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8773,550,22.5,474.5,player3,H,"[41.70966764384581, -72.58575982879059]",41.709668,-72.585760,41.709668,-72.585760,0.000000,0.000053,0.000000,5.518524,1.138306
9321,550,325.0,431.5,player4,H,"[41.70962481119451, -72.5857360061772]",41.709625,-72.585736,41.709624,-72.585735,0.000065,0.000045,6.830417,4.727982,9.158015
9869,550,1299.0,391.5,player5,F,"[41.70946369343943, -72.58569002263481]",41.709464,-72.585690,41.709463,-72.585690,0.000035,0.000013,3.624729,1.331829,33.302002
10417,550,676.0,203.5,player6,H,"[41.70959811845963, -72.58563740801932]",41.709598,-72.585637,41.709597,-72.585638,0.000067,0.000058,7.041890,6.064690,44.319801


df = pd.merge(df, df['distances_mile'].apply(pd.Series))

In [91]:
distance_df = []
for Name, group in df.groupby("Name"):
    print(Name)
    group['ft_traveled'] = group['haversine'].fillna(0).cumsum() * 5280
    distance_df.append(group)
df = pd.concat(distance_df).sort_values("frame")

Eric
Referee
ball
player1
player10
player11
player12
player13
player14
player15
player16
player17
player18
player19
player2
player3
player4
player5
player6
player7
player8
player9


In [92]:
df.tail(50)

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,haversine,haversine_smooth,mph,mph_smoothed,ft_to_ball,ft_traveled
1093,548,84.0,634.5,Referee,R,"[41.70963386328032, -72.58580796771736]",41.709634,-72.585808,41.709633,-72.585808,0.000010,0.000016,1.052401,1.651018,19.602533,79.439315
11142,548,791.5,77.0,player8,F,"[41.70959623168382, -72.58557698392029]",41.709596,-72.585577,41.709595,-72.585578,0.000043,0.000033,4.501374,3.404535,65.595836,173.940649
7127,548,241.5,37.0,player18,F,"[41.70970933510093, -72.58558041313984]",41.709709,-72.585580,41.709709,-72.585580,0.000031,0.000027,3.188691,2.859973,63.972752,71.742729
7675,548,3182.0,332.0,player19,F,"[41.709117922290545, -72.58559897055572]",41.709118,-72.585599,41.709118,-72.585599,0.000000,0.000007,0.000000,0.723389,83.080510,111.492213
4935,548,818.0,508.5,player14,F,"[41.70953324402831, -72.58574689889286]",41.709533,-72.585747,41.709533,-72.585747,0.000006,0.000012,0.628114,1.247119,15.352113,218.395984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8225,550,557.0,1077.5,player2,F,"[41.70951542212834, -72.58591130046862]",41.709515,-72.585911,41.709515,-72.585911,0.000015,0.000017,1.540657,1.737849,58.840016,153.151806
3841,550,912.0,471.0,player12,H,"[41.70952183703429, -72.58573121107183]",41.709522,-72.585731,41.709522,-72.585731,0.000000,0.000010,0.000000,1.048064,18.817265,138.288347
4389,550,12.5,454.5,player13,F,"[41.709672508759134, -72.58575350223977]",41.709673,-72.585754,41.709673,-72.585750,0.000248,0.000082,25.848710,8.517888,1.248471,130.083911
4937,550,814.5,506.5,player14,F,"[41.709534047997565, -72.58574633548234]",41.709534,-72.585746,41.709534,-72.585747,0.000021,0.000012,2.184745,1.244543,15.471495,218.621679


In [93]:
df[df['ft_traveled'] == df['ft_traveled'].max()]

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,haversine,haversine_smooth,mph,mph_smoothed,ft_to_ball,ft_traveled
1648,549,10.0,464.5,ball,N,"[41.709671282283075, -72.58575690471821]",41.709671,-72.585757,41.709671,-72.585755,0.000105,0.000073,10.917984,7.644886,0.0,327.263766
1649,550,10.0,464.5,ball,N,"[41.709671282283075, -72.58575690471821]",41.709671,-72.585757,41.709671,-72.585757,0.000000,0.000072,0.000000,7.526987,0.0,327.263766


### Plot to Pitch

In [94]:
quad_coords_out = {
    "lonlat": np.array([
        [41.709011, -72.585549], # Top Right
        [41.709691, -72.585688], # Top left
        [41.709500, -72.586105], # Bottom Left
        [41.709091, -72.586046] # Bottom Right
    ]),
    "pixel": np.array([
        [1540, 569], # top right
        [470, 569], # top left
        [666, 1299], # bottom left
        [1348, 1288] # bottom right
    ])
}
pm_out = PixelMapper(quad_coords_out["pixel"], quad_coords_out["lonlat"])
df['pixels_out'] = df.apply(lambda x: pm_out.lonlat_to_pixel((x['lon'], x['lat']))[0], axis=1)

In [95]:
pm_out.pixel_to_lonlat((1000,2000))

array([[ 41.70924948, -72.5864689 ]])

In [96]:
pm_out.lonlat_to_pixel((41.70924948, -72.5864689))

array([[1000.20808097, 2001.32018946]])

In [97]:
df['x_out'] = df['pixels_out'].str[0]
df['y_out'] = df['pixels_out'].str[1]

In [98]:
df

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,haversine,haversine_smooth,mph,mph_smoothed,ft_to_ball,ft_traveled,pixels_out,x_out,y_out
0,3,3381.5,941.0,Eric,F,"[41.70910594903356, -72.5858070933292]",41.709106,-72.585807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,"[1365.6055252615513, 915.2136355373904]",1365.605525,915.213636
10597,3,3241.0,1399.5,player8,F,"[41.70913930992976, -72.58592359169116]",41.709139,-72.585924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,"[1295.3770644058577, 1085.18451537831]",1295.377064,1085.184515
10418,3,1059.5,1608.5,player7,F,"[41.70941471559591, -72.58600711839543]",41.709415,-72.586007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,"[837.0362461926592, 1153.3542392454865]",837.036246,1153.354239
1096,3,NaN,NaN,ball,N,"[nan, nan]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,"[nan, nan]",NaN,NaN
9870,3,2955.0,1205.0,player6,H,"[41.70917583670772, -72.58588615310981]",41.709176,-72.585886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,"[1246.0853942821311, 1018.2377788213129]",1246.085394,1018.237779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8225,550,557.0,1077.5,player2,F,"[41.70951542212834, -72.58591130046862]",41.709515,-72.585911,41.709515,-72.585911,0.000015,0.000017,1.540657,1.737849,58.840016,153.151806,"[694.9406499135804, 971.9910284402345]",694.940650,971.991028
3841,550,912.0,471.0,player12,H,"[41.70952183703429, -72.58573121107183]",41.709522,-72.585731,41.709522,-72.585731,0.000000,0.000010,0.000000,1.048064,18.817265,138.288347,"[734.4037528175542, 688.4162667916476]",734.403753,688.416267
4389,550,12.5,454.5,player13,F,"[41.709672508759134, -72.58575350223977]",41.709673,-72.585754,41.709673,-72.585750,0.000248,0.000082,25.848710,8.517888,1.248471,130.083911,"[477.84105114175856, 678.535679874288]",477.841051,678.535680
4937,550,814.5,506.5,player14,F,"[41.709534047997565, -72.58574633548234]",41.709534,-72.585746,41.709534,-72.585747,0.000021,0.000012,2.184745,1.244543,15.471495,218.621679,"[710.3662058463946, 707.9490242242562]",710.366206,707.949024


In [99]:
import math

file_path = "".join(path.split(".")[:-1]).replace("labels/", "") + ".mp4"  # change to your own video path
# print(file_path)
vid = cv2.VideoCapture(file_path)
vid_height = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
vid_width = vid.get(cv2.CAP_PROP_FRAME_WIDTH)
# print(vid_height, vid_width)

img = cv2.imread('pitch.png')
height,width,layers=img.shape
print(width, height, layers)

ratio = width / vid_width, height / vid_height
print(ratio)

video=cv2.VideoWriter("".join(path.split(".")[:-1]).replace("labels/", "") + "_analysis.mp4" ,cv2.VideoWriter_fourcc(*'mp4v'),out_fps,(width,height))
# print(video)

for j in df.frame.unique():
    img = cv2.imread('pitch.png')
    image = cv2.putText(img, str(j), (100,100), fontFace=2, fontScale=.8, color=(0, 0, 0), thickness=2, lineType = cv2.LINE_AA)
    # print(img)
    frame = df[df['frame'] == j].to_dict(orient='records')
    for position in frame:
        # print(position)
        #image = cv2.circle(img, (j*5,100), radius=10, color=(255, 0, 0), thickness=-1)
        if set(["Name", "x_out", "y_out", "Team"]) <= set(position.keys()):
            if position["Team"] == "F":
                color = (255, 0, 0)
            elif position["Team"] == "H":
                color = (0, 0, 255)
            elif position['Name'] == "ball":
                color = (0, 0, 0)
            else:
                color = (0, 0, 0)
            position['Name'] = position['Name'] if position['Name'] != "ball" else "*"
            if (not math.isnan(position['x_out'])) and (not math.isnan(position['y_out'])):
                if math.isnan(position['mph_smoothed']):
                    position['mph_smoothed'] = 0
                if math.isnan(position['ft_to_ball']):
                    position['ft_to_ball'] = 0
                # print("Writing Vid")
                image = cv2.putText(img, "."+position['Name'], (int(position['x_out']), int(position['y_out'])), fontFace=2, fontScale=.8, color=color, thickness=2, lineType = cv2.LINE_AA)
                image = cv2.putText(img, str(round(position['mph_smoothed'])) + "mph | " + str(round(position['ft_to_ball'])) + "ft | " + str(round(position['ft_traveled'])) + "ft", (int(position['x_out']), int(position['y_out'] + 20)), fontFace=2, fontScale=.5, color=color, thickness=1, lineType = cv2.LINE_AA)
                # cv2.imshow("Output", image)
                # if cv2.waitKey(1) & 0xFF == ord('q'):
                #     break

    video.write(img)

cv2.destroyAllWindows()
video.release()

1998 1412 3
(0.5203125, 0.6537037037037037)


def click_event(event, x, y, flags, params):
    object_frame, frame, frame_n = params
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        # displaying the coordinates on the Shell
        # print(x, ' ', y)
        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        # if os.environ["clicked_x"] and os.environ["clicked_y"]:
        #     print("Changing Frame back to original")
        object_frame = copy.deepcopy(frame)
        cv2.putText(object_frame, "." + str(x) + ',' +
                    str(y), (x,y), font,
                    1, (255, 0, 0), 2)
        cv2.imshow('Scraping feed', object_frame)
        os.environ["clicked_x"] = str(x)
        # print("X" + os.environ["clicked_x"] + " - " + str(x))
        os.environ["clicked_y"] = str(y)
        instructions_frame = copy.deepcopy(frame)
        cv2.putText(instructions_frame, "You selected: " + os.environ["clicked_x"] + ", " + os.environ["clicked_y"], (100,500), cv2.FONT_HERSHEY_SIMPLEX, 5, (255, 0, 0), 5)
        cv2.imshow('Instructions feed', instructions_frame)
        print(frame_n, os.environ["clicked_x"], os.environ["clicked_y"])

import copy
import os

img = cv2.imread('pitch.png')
key_ind = "collecting"
while key_ind == "collecting":
    cv2.imshow('Data feed', img)
    cv2.setMouseCallback('Data feed', click_event, param=(img, img, 0))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
for i in range (1,5):
    cv2.waitKey(1)